In [1]:
from sentence_transformers import SentenceTransformer, util
import json
import torch
import numpy as np
import random
from tqdm.notebook import tqdm
# import os
import inflect
%config Completer.use_jedi = False

In [2]:
def pluralize(word):
    p = inflect.engine()
    return p.plural(word)

In [3]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1').cuda()

In [ ]:
def strip_prefix(caption):
    
    if caption.startswith("An image of a "):
        caption = caption.split("An image of a ")[-1]
    elif caption.startswith("An image of "):
        caption = caption.split("An image of ")[-1]
    elif caption.startswith("An image of... "):
        caption = caption.split("An image of... ")[-1]
        
#     if caption.startswith("Sure"):
#         print(caption)
#         print("&"*100)

    return caption

def strip_classname(caption):
    
    if caption.startswith("Possible category: "):
        caption = caption.split("Possible category: ")[-1] 
    elif caption.startswith("Sure!"):
        caption = caption.rsplit("\n",1)[-1].strip()
        
    if "\n" in caption:
        caption = caption.rsplit("\n",1)[-1]
        
    return caption

def get_classdesc(classnames, json_dict):
    descriptions = []
    for c in classnames:
        name = c.replace("+", " ").replace("_", " ")
        desc = " or ".join(json_dict[c])
        name_desc = "{} with {}".format(name, desc)
        descriptions.append(name_desc)
    return descriptions

### Caption - label similarity

In [4]:
def get_accuracy(caption_embeddings, class_embeddings, topk=5):
    similarity_matrix = util.cos_sim(caption_embeddings, class_embeddings)
    pseudoLabel = similarity_matrix.topk(topk, 1).indices    
    gt_label = torch.Tensor([id_to_classid[fid] for fid in all_flickrids]).view(-1,1).repeat(1,topk)
    matched_labels = (torch.Tensor(pseudoLabel) == torch.Tensor(gt_label))
    top1_acc = matched_labels[:,0].sum()/len(gt_label)
    topk_acc = matched_labels.any(1).sum()/len(gt_label)
    return top1_acc, topk_acc, similarity_matrix.argmax(1)

# def get_confident(caption_embeddings, class_embeddings, thres=.0025):
#     similarity_matrix = util.cos_sim(caption_embeddings, class_embeddings)
#     similarity_matrix = torch.nn.functional.softmax(similarity_matrix, dim=-1)
#     maxVal, pseudoLabel = similarity_matrix.max(1)
#     valid_inds = maxVal > thres
    
    
#     gt_label = torch.Tensor([id_to_classid[fid] for fid in all_flickrids])
#     matched_labels = (torch.Tensor(pseudoLabel) == torch.Tensor(gt_label))
#     matched_labels = matched_labels[valid_inds]
#     top1_acc = matched_labels.sum()/len(valid_inds)
# #     topk_acc = matched_labels.any(1).sum()/len(gt_label)
#     return top1_acc, sum(valid_inds)/len(similarity_matrix)*100

In [5]:
## category
meta = json.load(open("../metadata/geoplaces.json"))

In [6]:
cats = {int(c["category_id"]):c["category_name"] for c in meta["categories"]}
# cats = [cats[idx].replace(",","") for idx in range(68)]
cats = [cats[idx].replace("_indoor","").replace("_outdoor","").replace("_"," ") for idx in range(205)]
# cats = [cats[idx].replace("_"," ") for idx in range(205)]
# cats = [cats[idx].replace("+"," ").replace("_"," ") for idx in range(600)]

In [7]:
class_embeddings = model.encode(cats)

In [8]:
for src,dom in zip(["asia", "usa"], ["usa", "asia"]):
    geo = meta['{}_train'.format(dom)]
    id_to_cname = {ann["image_id"]:ann["class_name"] for ann in geo["annotations"]}
    id_to_classid = {ann["image_id"]:ann["category"] for ann in geo["annotations"]}
    id_to_blip = {m["image_id"]:" ".join([m["caption"], m["tags"].replace(","," "), m["description"]]) for m in geo["metadata"]}
#     id_to_blip = {m["image_id"]:m["tags"].replace(","," ") for m in geo["metadata"]}
#     id_to_cap = {e["image_id"]:e["llm_cap_llama_13b"] for e in geo["metadata"]}

#     cap_json = json.load(open("../metadata/condensed_caption_geoimnet_asia_13b.json"))['captions']
#     id_to_cap = {c["flickr_id"]:c["condensed_caption"] for c in cap_json}

    all_flickrids = list(id_to_blip.keys())
    all_captions = [id_to_blip[v] for v in all_flickrids]
    caption_embeddings = model.encode(all_captions)

    top1, top5, pseudo = get_accuracy(caption_embeddings, class_embeddings)
    print("{}:{:.2f}/{:.2f}".format(dom, top1*100, top5*100))
    
#     with open("../soft_labels/geoplaces_{}_{}_embedMatchPL.txt".format(src, dom), "w") as fh:
#         write_str = ""
#         for fid, pl in zip(all_flickrids, pseudo):
#             write_str += "{} {}\n".format(fid, pl)
#         fh.write(write_str)

usa:54.22/77.82
asia:46.75/70.48


In [ ]:
all_captions[100:120]

In [ ]:
for idx, cname in id_to_cname.items():
    tag_set = set(id_to_blip[idx].split(" "))
    cls = set(cname.split(","))
    cls = set(map(lambda v:v.replace(" ",""), cls))
    cls_aug = set(map(lambda v:pluralize(v), cls))
    cls = cls.union(cls_aug)

    common = tag_set.intersection(cls)
    
    if len(common) == 0:
        print("Tags:{}".format(tag_set))
        print("Class:{}".format(cls))
        print(common)
        print()

In [ ]:
cls

### Caption - Caption similarity

In [ ]:
import json
def get_similarity_acc(source_embed, target_embed, source_label, target_label, within=False, topks=[1,5]):
    
    topk = max(topks)
        
    if within:
        similarity_matrix = util.cos_sim(source_embed, source_embed)
        mostSimilar = similarity_matrix.topk(topk+1, 1).indices
        mostSimilar = mostSimilar[:,1:]
    else:
        similarity_matrix = util.cos_sim(source_embed, target_embed)
        mostSimilar = similarity_matrix.topk(topk, 1).indices
    
    similarLabels = torch.Tensor(target_label)[mostSimilar.long().reshape(-1)].reshape(-1, topk)
    source_label = torch.Tensor(source_label).view(-1,1).repeat(1,topk)
    
    matched_labels = (torch.Tensor(similarLabels) == torch.Tensor(source_label))
                      
    top1_acc = matched_labels[:,0].sum()/len(source_label)
    topk_acc = matched_labels.any(1).sum()/len(source_label)
        
    return top1_acc, topk_acc, similarLabels[:,0]

def get_similarity_acc_knn(source_embed, target_embed, source_label, target_label, within=False, K=1):
    
    topk = K
    
    similarity_matrix = util.cos_sim(source_embed, target_embed)
    mostSimilar = similarity_matrix.topk(topk, 1).indices
    
    similarLabels = torch.Tensor(target_label)[mostSimilar.long().reshape(-1)].reshape(-1, topk)
    similarLabels = torch.mode(similarLabels, dim=-1, keepdim=True).values
    source_label = torch.Tensor(source_label).view(-1,1)
    
    matched_labels = (torch.Tensor(similarLabels) == torch.Tensor(source_label))
                      
    top1_acc = matched_labels[:,0].sum()/len(source_label)
#     topk_acc = matched_labels.any(1).sum()/len(source_label)
        
    return top1_acc, top1_acc, similarLabels[:,0]

In [ ]:
for dataset in ['geoplaces', 'geoimnet', 'geoyfcc']:
    ## usa
    for src, tgt in zip(['asia','usa'],['usa','asia']):
        geo = json.load(open(f"../metadata/{dataset}.json"))
        source = geo['{}_train'.format(src)]
        id_to_cap_source = {m["image_id"]:" ".join([m["caption"], m["tags"].replace(","," "), m["description"]]) for m in source["metadata"]}
        id_to_label_source = {ann["image_id"]:ann["category"] for ann in source["annotations"]}

        ## asia
        target = geo['{}_train'.format(tgt)]
        id_to_cap_target = {m["image_id"]:" ".join([m["caption"], m["tags"].replace(","," "), m["description"]]) for m in target["metadata"]}
        id_to_label_target = {ann["image_id"]:ann["category"] for ann in target["annotations"]}

        source_flickrids = list(id_to_cap_source.keys())
        target_flickrids = list(id_to_cap_target.keys())

        source_captions = [id_to_cap_source[v] for v in source_flickrids]
        source_embeddings = model.encode(source_captions)
        source_labels = torch.tensor([id_to_label_source[v] for v in source_flickrids])

        target_captions = [id_to_cap_target[v] for v in target_flickrids]
        target_embeddings = model.encode(target_captions)
        target_labels = torch.Tensor([id_to_label_target[v] for v in target_flickrids])

        top1, top5, pseudo = get_similarity_acc_knn(target_embeddings, source_embeddings, target_labels, source_labels, K=3)

        print("{}({}->{}):{:.2f}/{:.2f}".format(dataset,src,tgt, top1*100, top5*100))
        
        assert len(target_flickrids) == len(pseudo)
#         with open("../soft_labels/{}_{}_{}_knn3PL.txt".format(dataset, src, tgt), "w") as fh:
#             write_str = ""
#             for fid, pl in zip(target_flickrids, pseudo):
#                 write_str += "{} {}\n".format(fid, pl)
#             fh.write(write_str)

In [ ]:
import json
geo = json.load(open(f"../metadata/domainnet.json"))
source = geo['{}_train'.format("real")]
id_to_cap_source = {m["image_id"]:m["blip2_cap"] for m in source["metadata"]}
id_to_label_source = {ann["image_id"]:ann["category"] for ann in source["annotations"]}

for tgt in ['clipart', 'sketch', 'painting']:    

    ## asia
    target = geo['{}_train'.format(tgt)]
    id_to_cap_target = {m["image_id"]:m["blip2_cap"] for m in target["metadata"]}
    id_to_label_target = {ann["image_id"]:ann["category"] for ann in target["annotations"]}

    source_flickrids = list(id_to_cap_source.keys())
    target_flickrids = list(id_to_cap_target.keys())

    source_captions = [id_to_cap_source[v] for v in source_flickrids]
    source_embeddings = model.encode(source_captions)
    source_labels = torch.tensor([id_to_label_source[v] for v in source_flickrids])

    target_captions = [id_to_cap_target[v] for v in target_flickrids]
    target_embeddings = model.encode(target_captions)
    target_labels = torch.Tensor([id_to_label_target[v] for v in target_flickrids])

    top1, top5, pseudo = get_similarity_acc_knn(target_embeddings, source_embeddings, target_labels, source_labels, K=201)

    print("({}->{}):{:.2f}/{:.2f}".format("real",tgt, top1*100, top5*100))

    assert len(target_flickrids) == len(pseudo)
#     with open("../soft_labels/{}_{}_{}_knn3PL.txt".format("domainnet", "real", tgt), "w") as fh:
#         write_str = ""
#         for fid, pl in zip(target_flickrids, pseudo):
#             write_str += "{} {}\n".format(fid, pl)
#         fh.write(write_str)

In [ ]:
get_similarity_acc(usa_embeddings, asia_embeddings, usa_labels, asia_labels)

In [ ]:
get_similarity_acc(asia_embeddings, usa_embeddings, asia_labels, usa_labels)

In [ ]:
similarity_matrix = util.cos_sim(asia_embeddings, usa_embeddings)

In [ ]:
ind = torch.sort(similarity_matrix, descending=True, dim=0).indices

In [ ]:
k_orderedNeighbors = torch.Tensor(asia_labels)[ind[:5]]

In [ ]:
k_orderedNeighbors.shape

In [ ]:
assigned_target_labels = torch.mode(k_orderedNeighbors, dim=0).values

In [ ]:
matched_labels = (torch.Tensor(assigned_target_labels) == torch.Tensor(usa_labels))
matched_labels.sum()/len(usa_labels)

## Nearest Captions

In [ ]:
##
meta = json.load(open("..//metadata/geoplaces.json"))

# cap_src = "llm_cap_llama_13b"
cap_src = "caption"

## usa
id_to_cap_usa = {e["image_id"]:e[cap_src] for e in meta['usa_train']['metadata']}
id_to_label_usa = {e["image_id"]:e["category"] for e in meta['usa_train']['annotations']}
id_to_class_usa = {e["image_id"]:e["class_name"] for e in meta['usa_train']['annotations']}

## asia
id_to_cap_asia = {e["image_id"]:e[cap_src] for e in meta['asia_train']['metadata']}
id_to_label_asia = {e["image_id"]:e["category"] for e in meta['asia_train']['annotations']}
id_to_class_asia = {e["image_id"]:e["class_name"] for e in meta['asia_train']['annotations']}

In [ ]:
usa_flickrids = list(id_to_cap_usa.keys())
asia_flickrids = list(id_to_cap_asia.keys())

usa_captions = [id_to_cap_usa[v] for v in usa_flickrids]
usa_embeddings = model.encode(usa_captions)

asia_captions = [id_to_cap_asia[v] for v in asia_flickrids]
asia_embeddings = model.encode(asia_captions)

In [ ]:
usa_labels = torch.tensor([id_to_label_usa[v] for v in usa_flickrids])
asia_labels = torch.tensor([id_to_label_asia[v] for v in asia_flickrids])

In [ ]:
usa_classes = [id_to_class_usa[v] for v in usa_flickrids]
asia_classes = [id_to_class_asia[v] for v in asia_flickrids]

In [ ]:
similarity = util.cos_sim(asia_embeddings, usa_embeddings)

K = 5
mostSimilar = similarity.topk(K, dim=-1).indices

mostSimilar.shape

mostSimilarLabels = usa_labels[mostSimilar]
pseudoLabels = torch.mode(mostSimilarLabels, dim=-1).values

matched_labels = (torch.Tensor(pseudoLabels) == torch.Tensor(asia_labels))
matched_labels.sum()/len(asia_labels)

In [ ]:
similarity = util.cos_sim(usa_embeddings, asia_embeddings)

K = 5
mostSimilar = similarity.topk(K, dim=-1).indices

mostSimilar.shape

mostSimilarLabels = asia_labels[mostSimilar]
pseudoLabels = torch.mode(mostSimilarLabels, dim=-1).values

matched_labels = (torch.Tensor(pseudoLabels) == torch.Tensor(usa_labels))
matched_labels.sum()/len(usa_labels)

In [ ]:
print(get_similarity_acc(asia_embeddings, usa_embeddings, asia_labels, usa_labels))
print(get_similarity_acc(usa_embeddings, asia_embeddings, usa_labels, asia_labels))

In [ ]:
matched_labels = (torch.Tensor(pseudoLabels) == torch.Tensor(asia_labels))
matched_labels.sum()/len(asia_labels)

In [ ]:
for idx in np.random.choice(np.arange(1e5), 20):
    index = int(idx)
    otherIdx = mostSimilar[index][0]
    print("[{}]{}:{}::{}:{}\n".format(int(asia_classes[index] == usa_classes[otherIdx]),\
                                      asia_captions[index],\
                               asia_classes[index],\
                               usa_captions[otherIdx],
                               usa_classes[otherIdx]))
    